In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

In [23]:
df_raw = pd.read_csv('kredit.csv')

In [24]:
df = df_raw.copy(deep=True)

In [58]:
df['altersgruppe'] = np.nan

lst = [df]

for col in lst:
    col.loc[(col['alter'] > 18) & (col['alter'] <= 25), 'altersgruppe'] = 'Young Adults'
    col.loc[(col['alter'] > 25) & (col['alter'] <= 39), 'altersgruppe'] = 'Adults'
    col.loc[(col['alter'] > 39) & (col['alter'] <= 59), 'altersgruppe'] = 'Senior'
    col.loc[col['alter'] > 59, 'altersgruppe'] = 'Elder' 
    
df.head()

,kredit,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,altersgruppe
0,1,1,18,4,2,1049,1,2,4,2,...,2,21,3,1,1,3,1,1,1,Young Adults
1,1,1,9,4,0,2799,1,3,2,3,...,1,36,3,1,2,3,2,1,1,Adults
2,1,2,12,2,9,841,2,4,2,2,...,1,23,3,1,1,2,1,1,1,Young Adults
3,1,1,12,4,0,2122,1,3,3,3,...,1,39,3,1,2,2,2,1,2,Adults
4,1,1,12,4,0,2171,1,3,4,3,...,2,38,1,2,2,2,1,1,2,Adults


In [25]:
df['zahlung pro monat'] = df['hoehe'] / df['laufzeit']

In [26]:
df.head()

,kredit,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,zahlung pro monat
0,1,1,18,4,2,1049,1,2,4,2,...,2,21,3,1,1,3,1,1,1,58.277778
1,1,1,9,4,0,2799,1,3,2,3,...,1,36,3,1,2,3,2,1,1,311.000000
2,1,2,12,2,9,841,2,4,2,2,...,1,23,3,1,1,2,1,1,1,70.083333
3,1,1,12,4,0,2122,1,3,3,3,...,1,39,3,1,2,2,2,1,2,176.833333
4,1,1,12,4,0,2171,1,3,4,3,...,2,38,1,2,2,2,1,1,2,180.916667


In [6]:
X

NameError: name 'X' is not defined

In [78]:
y

0      1
1      1
2      1
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: kredit, Length: 1000, dtype: int64

In [9]:
#logit
from sklearn.linear_model import LogisticRegression


X = df.drop('kredit', axis=1)
y = df['kredit']

cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
log_model = LogisticRegression()

space = {'C':np.linspace(0.1,5.0),'solver':['newton-cg', 'lbfgs', 'sag', 'saga','liblinear'],'penalty':['l1','l2']}

search = GridSearchCV(estimator= log_model, param_grid= space, 
                      scoring='accuracy', n_jobs=1, cv=cv_inner, refit=True, verbose=2)
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(search, X=X,y= y, scoring='accuracy', cv=cv_outer, n_jobs=-1)

print(scores.mean())

[0.81 0.73 0.74 0.68 0.77 0.77 0.71 0.82 0.71 0.75]


In [11]:
scores.mean()

0.7489999999999999

In [19]:
#KNN
from sklearn.neighbors import KNeighborsClassifier


X = df.drop('kredit', axis=1)
y = df['kredit']

cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
knn_model = KNeighborsClassifier()

space = {'n_neighbors' : [3,5,7,9],
    'weights' : ['uniform','distance'],
    'p' : [1,2],
    'metric' : ['euclidean', 'manhattan', 'minkowski']}

search = GridSearchCV(estimator= knn_model, param_grid= space, scoring='accuracy', n_jobs=1, cv=cv_inner, refit=True, verbose=2)
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(search, X=X,y= y, scoring='accuracy', cv=cv_outer, n_jobs=-1)

print(scores)

[0.74 0.69 0.66 0.6  0.76 0.68 0.64 0.71 0.69 0.68]


In [20]:
scores.mean()

0.6849999999999999

In [27]:
#SVC

from sklearn.svm import SVC

X = df.drop('kredit', axis=1)
y = df['kredit']

cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)
svc_model = SVC(probability = True, max_iter = 2000)

space = {'kernel' : ['linear','rbf'],
    'C' : [1,10,100,1000],
    'gamma' : [1,0.1,0.001,0.0001]}

search = GridSearchCV(estimator= svc_model, param_grid= space, scoring='accuracy', n_jobs=1, cv=cv_inner, refit=True, verbose=2)
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(search, X=X,y= y, scoring='accuracy', cv=cv_outer, n_jobs=-1)

print(scores)

[0.73 0.71 0.65 0.61 0.76 0.68 0.66 0.71 0.71 0.73]


In [28]:
scores.mean()

0.695

NameError: name 'best_params_' is not defined

In [ ]:
NUM_TRIALS = 30

space = {'C':np.linspace(0.1,5.0),'solver':['newton-cg', 'lbfgs', 'sag', 'saga','liblinear'],'penalty':['l1','l2']}
logit = LogisticRegression()

non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    
    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=logit, param_grid=space, cv=inner_cv)
    clf.fit(X, y)
    non_nested_scores[i] = clf.best_score_
    
    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv,n_jobs=-1)
    nested_scores[i] = nested_score.mean()
    
print(non_nested_scores)
print(nested_scores)